In [7]:
from neon.backends import gen_backend
from neon.data import DataIterator, Text, load_text
from neon.initializers import Uniform, GlorotUniform
from neon.layers import GeneralizedCost, LSTM, Affine, Dropout, LookupTable, RecurrentSum
from neon.models import Model
from neon.optimizers import Adagrad
from neon.transforms import Logistic, Tanh, Softmax, CrossEntropyMulti, Accuracy
from neon.callbacks.callbacks import Callbacks
from neon.util.argparser import NeonArgparser

from collections import defaultdict
import numpy as np
import scipy as sp
import cPickle as pickle
import time
import os
data_root = os.path.expanduser("~") + '/data/CSE255/'

from nltk.stem.porter import PorterStemmer

%matplotlib inline
import matplotlib.pyplot as plt

import string
stemmer = PorterStemmer()
punctuation = set(string.punctuation)


NameError: name 'string' is not defined

In [3]:
start_time = time.time()
all_data = pickle.load(open(data_root + "all_data.pickle", "rb"))
test_data = pickle.load(open(data_root + "helpful_data.pickle", "rb"))
all_and_test_data = all_data + test_data
print(time.time() - start_time)

18.2088780403


In [5]:
all_size = len(all_data)
train_size = 900000
# train_size = all_size
valid_size = 100000
train_data = all_data[:train_size]
valid_data = all_data[all_size - valid_size:]

In [81]:
# load dictionary
imdb_word_to_idx = pickle.load(open(data_root + "imdb.dict.pkl", "rb"))
imdb_idx_to_word = dict()
for word, idx in imdb_word_to_idx.iteritems():
    imdb_idx_to_word[idx] = word

In [139]:
# convert data to word indecies
features = []
ratios = []

for idx, d in enumerate(all_data):
    if idx % 10000 == 0:
        print("%d of %d" % (idx, len(all_data)))
    
    if float(d['helpful']['outOf']) == 0:
        continue
        
    review_text = d['summary'].split() + d['reviewText'].split()
    
    # pre-process text
    review_text = [w.lower() for w in review_text if w not in punctuation]
    review_text = ["".join(c for c in w if c not in punctuation) for w in review_text]
    # review_text = [stemmer.stem(w) for w in review_text]
    review_text = [w for w in review_text if w in imdb_word_to_idx]
    review_text_idx = [imdb_word_to_idx[w] for w in review_text]
    review_text_idx = np.array(review_text_idx).astype(int)
    
    # generate text feature
    feature = review_text_idx
    ratio = float(d['helpful']['nHelpful']) / float(d['helpful']['outOf'])
    
    # append
    features.append(feature)
    ratios.append(ratio)

0 of 1000000
10000 of 1000000
20000 of 1000000
30000 of 1000000
40000 of 1000000
50000 of 1000000
60000 of 1000000
70000 of 1000000
80000 of 1000000
90000 of 1000000
100000 of 1000000
110000 of 1000000
120000 of 1000000
130000 of 1000000
140000 of 1000000
150000 of 1000000
160000 of 1000000
170000 of 1000000
180000 of 1000000
190000 of 1000000
200000 of 1000000
210000 of 1000000
220000 of 1000000
230000 of 1000000
240000 of 1000000
250000 of 1000000
260000 of 1000000
270000 of 1000000
280000 of 1000000
290000 of 1000000
300000 of 1000000
310000 of 1000000
320000 of 1000000
330000 of 1000000
340000 of 1000000
350000 of 1000000
360000 of 1000000
370000 of 1000000
380000 of 1000000
390000 of 1000000
400000 of 1000000
410000 of 1000000
420000 of 1000000
430000 of 1000000
440000 of 1000000
450000 of 1000000
460000 of 1000000
470000 of 1000000
480000 of 1000000
490000 of 1000000
500000 of 1000000
510000 of 1000000
520000 of 1000000
530000 of 1000000
540000 of 1000000
550000 of 1000000
560000

In [140]:
# pickle.dump((features, ratios), open(data_root + "train_text_data.pickle", "wb"), 
#             protocol = pickle.HIGHEST_PROTOCOL)

In [6]:
(features, ratios) = pickle.load(open(data_root + "train_text_data.pickle", "rb"))
# pickle.dump((features[:100000], ratios[:100000]), 
#             open(data_root + "train_text_data_100000.pickle", "wb"), 
#             protocol = pickle.HIGHEST_PROTOCOL)